In [7]:
import os
import re
import numpy as np
import tensorflow as tf

In [8]:
import sys
base_dir = "keras-facenet/"
sys.path.append(base_dir + '/code/')
from inception_resnet_v1 import *

tf_model_dir = base_dir + 'model/tf/'
npy_weights_dir = base_dir + 'model/keras/npy_weights/'
weights_dir = base_dir + 'model/keras/weights/'
model_dir = base_dir + 'model/keras/model/'

weights_filename = 'facenet_keras_weights.h5'
model_filename = 'facenet_keras.h5'

os.makedirs(npy_weights_dir, exist_ok=True)
os.makedirs(weights_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

In [9]:
# regex for renaming the tensors to their corresponding Keras counterpart
re_repeat = re.compile(r'Repeat_[0-9_]*b')
re_block8 = re.compile(r'Block8_[A-Za-z]')

def get_filename(key):
    filename = str(key)
    filename = filename.replace('/', '_')
    filename = filename.replace('InceptionResnetV1_', '')

    # remove "Repeat" scope from filename
    filename = re_repeat.sub('B', filename)

    if re_block8.match(filename):
        # the last block8 has different name with the previous 5 occurrences
        filename = filename.replace('Block8', 'Block8_6')

    # from TF to Keras naming
    filename = filename.replace('_weights', '_kernel')
    filename = filename.replace('_biases', '_bias')

    return filename + '.npy'


def extract_tensors_from_checkpoint_file(filename, output_folder):
    reader = tf.compat.v1.train.NewCheckpointReader(filename)

    for key in reader.get_variable_to_shape_map():
        # not saving the following tensors
        if key == 'global_step':
            continue
        if 'AuxLogit' in key:
            continue

        # convert tensor name into the corresponding Keras layer weight name and save
        path = os.path.join(output_folder, get_filename(key))
        arr = reader.get_tensor(key)
        np.save(path, arr)

In [10]:
extract_tensors_from_checkpoint_file(tf_model_dir+"model-20180408-102900.ckpt-90", npy_weights_dir);

In [11]:
model = InceptionResNetV1Norm(classes=512)

In [12]:
model.summary()

Model: "inception_resnet_v1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv2d_1a_3x3 (Conv2D)          (None, 79, 79, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
Conv2d_1a_3x3_BatchNorm (BatchN (None, 79, 79, 32)   96          Conv2d_1a_3x3[0][0]              
__________________________________________________________________________________________________
Conv2d_1a_3x3_Activation (Activ (None, 79, 79, 32)   0           Conv2d_1a_3x3_BatchNorm[0][0]    
________________________________________________________________________________

In [6]:
print('Loading numpy weights from', npy_weights_dir)
for layer in model.layers:
    if layer.weights:
        weights = []
        for w in layer.weights:
            print(w.name)
            weight_name = os.path.basename(w.name).replace(':0', '')
            weight_file = layer.name + '_' + weight_name + '.npy'
            weight_arr = np.load(os.path.join(npy_weights_dir, weight_file))
            weights.append(weight_arr)
        layer.set_weights(weights)

print('Saving weights...')
model.save_weights(os.path.join(weights_dir, weights_filename))
print('Saving model...')
model.save(os.path.join(model_dir, model_filename))

Loading numpy weights from keras-facenet/model/keras/npy_weights/
Conv2d_1a_3x3/kernel:0
Conv2d_1a_3x3_BatchNorm/beta:0
Conv2d_1a_3x3_BatchNorm/moving_mean:0
Conv2d_1a_3x3_BatchNorm/moving_variance:0
Conv2d_2a_3x3/kernel:0
Conv2d_2a_3x3_BatchNorm/beta:0
Conv2d_2a_3x3_BatchNorm/moving_mean:0
Conv2d_2a_3x3_BatchNorm/moving_variance:0
Conv2d_2b_3x3/kernel:0
Conv2d_2b_3x3_BatchNorm/beta:0
Conv2d_2b_3x3_BatchNorm/moving_mean:0
Conv2d_2b_3x3_BatchNorm/moving_variance:0
Conv2d_3b_1x1/kernel:0
Conv2d_3b_1x1_BatchNorm/beta:0
Conv2d_3b_1x1_BatchNorm/moving_mean:0
Conv2d_3b_1x1_BatchNorm/moving_variance:0
Conv2d_4a_3x3/kernel:0
Conv2d_4a_3x3_BatchNorm/beta:0
Conv2d_4a_3x3_BatchNorm/moving_mean:0
Conv2d_4a_3x3_BatchNorm/moving_variance:0
Conv2d_4b_3x3/kernel:0
Conv2d_4b_3x3_BatchNorm/beta:0
Conv2d_4b_3x3_BatchNorm/moving_mean:0
Conv2d_4b_3x3_BatchNorm/moving_variance:0
Block35_1_Branch_2_Conv2d_0a_1x1/kernel:0
Block35_1_Branch_2_Conv2d_0a_1x1_BatchNorm/beta:0
Block35_1_Branch_2_Conv2d_0a_1x1_Batch